In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1765289937462_0002,pyspark,idle,Link,Link,None,
5,application_1765289937462_0006,pyspark,idle,Link,Link,None,
6,application_1765289937462_0007,pyspark,idle,Link,Link,None,
9,application_1765289937462_0010,pyspark,dead,Link,Link,None,


In [8]:
from pyspark.sql import SparkSession
import csv
from io import StringIO

def parse_csv(line):
    return next(csv.reader([line]))
sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .getOrCreate() \
    .sparkContext

rdd1 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv")\
            .map(parse_csv)

rdd2 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv")\
            .map(parse_csv)

crime_rdd_raw = rdd1.union(rdd2)

# header = crime_rdd_raw.first()
# crime_rdd_raw = crime_rdd_raw.filter(lambda line: line != header)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
import time

def parse_age(s):
    try:
        age = int(s)
        return age if age > 0 else None
    except:
        return None

def age_bucket(age):
    if age is None:
        return None
    if age < 18:
        return "Children"
    elif age <= 24:
        return "Young Adults"
    elif age <= 64:
        return "Adults"
    else:
        return "Elderly"

start = time.time()
agg_buckets = (
    crime_rdd_raw
    .filter(lambda r: len(r) > 11 and r[9] and "aggravated assault" in r[9].lower())
    .map(lambda r: (age_bucket(parse_age(r[11])), 1))
    .filter(lambda x: x[0] is not None)
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)

for group, cnt in agg_buckets.collect():
    print(group, cnt)

end = time.time()
print("Execution time:", end - start, "seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Adults 121660
Young Adults 33758
Children 10904
Elderly 6011
Execution time: 25.0686616897583 seconds

In [7]:
agg_buckets = (
    crime_rdd_raw
    # keep only aggravated assaults
    .filter(lambda r: len(r) > 11 and r[9] is not None and "aggravated assault" in r[9].lower())
    # map to (bucket, 1)
    .map(lambda r: (age_bucket(parse_age(r[11])), 1))
    # drop rows with no valid bucket
    .filter(lambda x: x[0] is not None)
    # count per bucket
    .reduceByKey(lambda a, b: a + b)
    # sort by count descending
    .sortBy(lambda x: x[1], ascending=False)
)

result = agg_buckets.collect()

for group, cnt in result:
    print(group, cnt)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 4.0 failed 4 times, most recent failure: Lost task 1.3 in stage 4.0 (TID 62) (ip-192-168-1-91.eu-central-1.compute.internal executor 9): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1765289937462_0011/container_1765289937462_0011_01_000014/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/mnt1/yarn/usercache/livy/appcache/application_1765289937462_0011/container_1765289937462_0011_01_000014/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/mnt1/yarn/usercache/livy/appcache/application_1765289937462_0011/container_1765289937462_0011_01_000001/pyspark.zip/pyspark/rdd.py", line 5434, in pipeline_func
  File "/mnt1/yarn/userca